In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from efficientnet_pytorch import EfficientNet

## DATASET

In [ ]:
class KeypointsDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform):
        self.annotation = pd.read_csv(csv_file).dropna(axis=0, how='any')
        self.annotation = self.annotation.reset_index(drop=True)
        self.transform = transform
       
    def __len__(self):
        return len(self.annotation)
    
    def __getitem__(self, index):
        img = self.annotation.iloc[index]["Image"]
        image = np.array(img.split()).reshape(96, 96)
        image = np.repeat(image.reshape(96,96,1),3,2).astype(np.uint8)
        keypoints = self.annotation.iloc[index].to_numpy()[:-1].reshape(-1,2)
        augmentations = self.transform(image=image, keypoints=keypoints)
        image = augmentations["image"]
        keypoints = np.array(augmentations["keypoints"]).reshape(-1)
        return image, keypoints
    
    def __show_img_annotated__(self, index):
        img = self.annotation.iloc[index]["Image"]
        image = np.array(img.split()).astype(np.uint8).reshape(96, 96)
        #image = Image.fromarray(image, 'L')
        plt.imshow(image, cmap="gray")
        keypoints = self.annotation.iloc[index].to_numpy()[:-1].reshape(-1,2)
        plt.scatter(keypoints[:,0], keypoints[:,1], c="white")
        plt.show()

In [ ]:
transform = A.Compose([A.Resize(width=96, height=96),
                       ToTensorV2(),
                      ], keypoint_params=A.KeypointParams(format="xy", remove_invisible=False))

In [ ]:
dataset = KeypointsDataset("trainingdataset.csv", transform)

In [ ]:
dataset.__show_img_annotated__(590)

In [ ]:
## split data
train_size = int(np.floor(len(dataset)*0.8))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

## MODEL: efficientNet

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Sequential(nn.Linear(1280, 30))
model = model.to(device)

## TRAINING

In [ ]:
# Hyperparameters
EPOCH = 10
LR = 1e-4
BATCH_SIZE = 64
loss_function = nn.MSELoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, BATCH_SIZE, shuffle=True)

In [ ]:
def train(model):
    for epoch in range(100):
        losses = []
        num_examples = 0
        for batch in tqdm(train_loader):
            X_batch ,Y_batch = batch[0].view(-1,3,96,96).to(device), batch[1].to(device)
            output = model(X_batch.float())
            loss = loss_function(output, Y_batch.float())
            num_examples += torch.numel(Y_batch)
            losses.append(loss)
            model.zero_grad()
            loss.backward()
            optimizer.step()
           
        print(f"Loss at epoch {epoch} is {(sum(losses)/num_examples)}")

In [ ]:
train(model)